In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import callbacks
import os
import cv2
import string

Using TensorFlow backend.


In [3]:
symbols = string.ascii_lowercase + '0123456789'
num_symbols = len(symbols)
img_shape = (50,200,1)

In [4]:
def preprocess_data():
    n_samples = len(os.listdir('samples/'))
    X = np.zeros((n_samples,50,200,1))
    y = np.zeros((5,n_samples,num_symbols))
    
    for i, pic in enumerate(os.listdir('samples/')):
        img = cv2.imread(os.path.join('samples/',pic),cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4]
        if len(pic_target) < 6:
            img = img/255.0
            img = np.reshape(img,(50,200,1))
            targs = np.zeros((5,num_symbols))
            for j,l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j,ind] = 1
            X[i] = img
            y[:,i]=targs
    return X,y

X,y = preprocess_data()
X_train, y_train = X[:970],y[:,:970]
X_test, y_test = X[970:],y[:,970:]

In [5]:
def create_model():
    img = layers.Input(shape=img_shape)
    conv1 = layers.Conv2D(16,(3,3),padding='same',activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)
    conv2 = layers.Conv2D(32,(3,3),padding='same',activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(mp1)
    conv3 = layers.Conv2D(32,(3,3),padding = 'same',activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D(padding='same')(bn)
    
    flat = layers.Flatten()(mp3)
    outs = []
    for _ in range(5):
        dens1 = layers.Dense(64,activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols,activation='sigmoid')(drop)
        
        outs.append(res)
        
    model = Model(img,outs)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

        

In [6]:
model = create_model()
model.summary()

W0829 21:20:09.657624  8528 deprecation_wrapper.py:119] From C:\Users\hp\.conda\envs\tensor_test\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0829 21:20:09.671616  8528 deprecation_wrapper.py:119] From C:\Users\hp\.conda\envs\tensor_test\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 21:20:09.675576  8528 deprecation_wrapper.py:119] From C:\Users\hp\.conda\envs\tensor_test\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0829 21:20:09.687575  8528 deprecation_wrapper.py:119] From C:\Users\hp\.conda\envs\tensor_test\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0829 21:20:09.727470  8528 depreca

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50, 200, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 50, 200, 16)  160         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 25, 100, 16)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 13, 50, 16)   0           max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_3 (

In [7]:
hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], batch_size=32, epochs=30,verbose=1, validation_split=0.2)

W0829 21:20:16.586280  8528 deprecation.py:323] From C:\Users\hp\.conda\envs\tensor_test\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 776 samples, validate on 194 samples
Epoch 1/30
776/776 [==============================] - 11s 14ms/step - loss: 17.3904 - dense_2_loss: 3.4135 - dense_4_loss: 3.4778 - dense_6_loss: 3.4341 - dense_8_loss: 3.4733 - dense_10_loss: 3.5917 - dense_2_acc: 0.0838 - dense_4_acc: 0.0657 - dense_6_acc: 0.0696 - dense_8_acc: 0.0399 - dense_10_acc: 0.0387 - val_loss: 16.2329 - val_dense_2_loss: 3.3963 - val_dense_4_loss: 3.2109 - val_dense_6_loss: 3.0602 - val_dense_8_loss: 3.1872 - val_dense_10_loss: 3.3783 - val_dense_2_acc: 0.0000e+00 - val_dense_4_acc: 0.0309 - val_dense_6_acc: 0.0979 - val_dense_8_acc: 0.0361 - val_dense_10_acc: 0.0567
Epoch 2/30
776/776 [==============================] - 1s 2ms/step - loss: 15.8828 - dense_2_loss: 3.0598 - dense_4_loss: 3.1887 - dense_6_loss: 3.1672 - dense_8_loss: 3.1609 - dense_10_loss: 3.3062 - dense_2_acc: 0.0863 - dense_4_acc: 0.0709 - dense_6_acc: 0.0812 - dense_8_acc: 0.0619 - dense_10_acc: 0.0490 - val_loss: 16.3091 - val_dense_2_loss: 3.8

Epoch 15/30
776/776 [==============================] - 1s 2ms/step - loss: 7.6452 - dense_2_loss: 0.7367 - dense_4_loss: 1.4023 - dense_6_loss: 1.7178 - dense_8_loss: 1.6708 - dense_10_loss: 2.1177 - dense_2_acc: 0.7320 - dense_4_acc: 0.4523 - dense_6_acc: 0.4613 - dense_8_acc: 0.4265 - dense_10_acc: 0.1920 - val_loss: 16.2750 - val_dense_2_loss: 10.1880 - val_dense_4_loss: 1.1899 - val_dense_6_loss: 1.5487 - val_dense_8_loss: 1.4848 - val_dense_10_loss: 1.8637 - val_dense_2_acc: 0.0000e+00 - val_dense_4_acc: 0.7216 - val_dense_6_acc: 0.6237 - val_dense_8_acc: 0.5309 - val_dense_10_acc: 0.2577
Epoch 16/30
776/776 [==============================] - 1s 2ms/step - loss: 6.9988 - dense_2_loss: 0.6564 - dense_4_loss: 1.3045 - dense_6_loss: 1.5254 - dense_8_loss: 1.5606 - dense_10_loss: 1.9520 - dense_2_acc: 0.7835 - dense_4_acc: 0.4729 - dense_6_acc: 0.5039 - dense_8_acc: 0.4356 - dense_10_acc: 0.2448 - val_loss: 15.4142 - val_dense_2_loss: 9.7846 - val_dense_4_loss: 1.0621 - val_dense_6_lo

Epoch 29/30
776/776 [==============================] - 1s 2ms/step - loss: 3.3928 - dense_2_loss: 0.2846 - dense_4_loss: 0.7431 - dense_6_loss: 0.6660 - dense_8_loss: 0.7263 - dense_10_loss: 0.9727 - dense_2_acc: 0.8879 - dense_4_acc: 0.6933 - dense_6_acc: 0.7500 - dense_8_acc: 0.7165 - dense_10_acc: 0.6134 - val_loss: 13.8926 - val_dense_2_loss: 10.0323 - val_dense_4_loss: 0.8914 - val_dense_6_loss: 1.1930 - val_dense_8_loss: 0.9188 - val_dense_10_loss: 0.8572 - val_dense_2_acc: 0.0979 - val_dense_4_acc: 0.8196 - val_dense_6_acc: 0.7268 - val_dense_8_acc: 0.7474 - val_dense_10_acc: 0.7784
Epoch 30/30
776/776 [==============================] - 1s 2ms/step - loss: 3.2076 - dense_2_loss: 0.2690 - dense_4_loss: 0.6825 - dense_6_loss: 0.6426 - dense_8_loss: 0.6786 - dense_10_loss: 0.9348 - dense_2_acc: 0.8956 - dense_4_acc: 0.7294 - dense_6_acc: 0.7397 - dense_8_acc: 0.7204 - dense_10_acc: 0.6482 - val_loss: 14.2322 - val_dense_2_loss: 10.3638 - val_dense_4_loss: 0.9613 - val_dense_6_loss:

In [8]:
from keras.models import model_from_json


In [9]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# later...

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")



Saved model to disk
Loaded model from disk


In [10]:
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = img / 255.0
    else:
        print("Not detected");
    res = np.array(loaded_model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, 36))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        #probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += symbols[l]
    return print("Result:",capt)#, sum(probs) / 5

In [11]:
from pyfiglet import Figlet
def cli_test():
    f = Figlet(font='slant')
    print(f.renderText('Captcha???? Hold My Beer!!'))
    file = input("Get the Captcha!=")
    print(predict(file))

In [23]:
cli_test()

   ______            __       __         ___  ___  ___  ___ 
  / ____/___ _____  / /______/ /_  ____ /__ \/__ \/__ \/__ \
 / /   / __ `/ __ \/ __/ ___/ __ \/ __ `// _/ / _/ / _/ / _/
/ /___/ /_/ / /_/ / /_/ /__/ / / / /_/ //_/  /_/  /_/  /_/  
\____/\__,_/ .___/\__/\___/_/ /_/\__,_/(_)  (_)  (_)  (_)   
          /_/                                               
    __  __      __    __   __  ___         ____                 ____
   / / / /___  / /___/ /  /  |/  /_  __   / __ )___  ___  _____/ / /
  / /_/ / __ \/ / __  /  / /|_/ / / / /  / __  / _ \/ _ \/ ___/ / / 
 / __  / /_/ / / /_/ /  / /  / / /_/ /  / /_/ /  __/  __/ /  /_/_/  
/_/ /_/\____/_/\__,_/  /_/  /_/\__, /  /_____/\___/\___/_/  (_|_)   
                              /____/                                

Get the Captcha!=test3.png
Result: gnbn4
None
